In [ ]:
import os
import json

# Define the path to your dataset
dataset_path = 'path/to/your/dataset'

# Initialize a dictionary to store image paths and their corresponding labels
image_labels = {}

# Loop over each class directory
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    
    if os.path.isdir(class_dir):  # Ensure it's a directory
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            
            if os.path.isfile(img_path):  # Ensure it's a file
                image_labels[img_path] = class_name

# Optionally, save the labels to a JSON file for future use
with open('image_labels.json', 'w') as json_file:
    json.dump(image_labels, json_file, indent=4)

print("Labels generated and saved to image_labels.json")